In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
import os
from dotenv import load_dotenv



In [2]:
load_dotenv()

True

In [3]:
ARCHIVO_SALIDA = os.getenv("ARCHIVO_SALIDA")
ARCHIVO_RAW= os.getenv("ARCHIVO_RAW")

In [4]:
df_raw=pd.read_parquet(ARCHIVO_RAW)

In [5]:
df_raw.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [6]:
df=df_raw.copy()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


# ANALIZO SI EXISTEN DUPLICADOS GLOBALES

In [8]:
print(df.duplicated().sum())

98


In [9]:
#Elimino los duplicados
df = df.drop_duplicates()

In [10]:
#Compruebo si hay ids duplicados:
print(df.id_reserva.duplicated().sum())

0


# TRANSFORMACIÓN DE LAS COLUMNAS TIPO FECHA

In [11]:
def tran_data(lista_col, dataframe):
    """_summary_

    Args:
        lista_col (_type_): _description_
        dataframe (_type_): _description_
    """
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")

In [12]:
col_fechas=["fecha_reserva","inicio_estancia","final_estancia"]
tran_data(col_fechas, df)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  14925 non-null  datetime64[ns]
 8   final_estancia   14925 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

# TRANSFORMACIÓN DE LA COLUMNA estrellas  

In [13]:
df["estrellas"] = df["estrellas"].astype('Int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  14925 non-null  datetime64[ns]
 8   final_estancia   14925 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   Int64         
 13  ciudad           15000 non-null  object        
dtypes: Int64(1), bool(1), datetime64[ns](3), fl

In [14]:
df["nombre_hotel"].value_counts()

nombre_hotel
                          5172
Hotel Sol y Luna           557
Hotel Monte Verde          550
Hotel Luz de Madrid        536
Hotel Brisas del Mar       534
Hotel Mirador Real         533
Hotel Los Almendros        533
Hotel Puerta del Cielo     526
Hotel Camino del Sol       518
Gran Hotel Madrid          518
Hotel Costa Azul           515
Hotel Las Estrellas        514
Hotel Torre Dorada         512
Hotel Rincón Sereno        512
Hotel Vista Alegre         511
Hotel Encanto Real         511
Palacio del Sol            510
Hotel Jardines del Rey     502
Hotel Maravilla Real       473
Hotel Palacio Imperial     463
Name: count, dtype: int64

In [15]:
df_competencia = df[df.competencia == True]

In [16]:
#aunque no aparezca como nulos, en realidad el nombre del hotel está vacío:
df_competencia.nombre_hotel.value_counts()



nombre_hotel
    5172
Name: count, dtype: int64

In [17]:
#aunque no aparezca como nulos, en realidad el nombre de la ciudad está vacío:
df_competencia.ciudad.value_counts()

ciudad
    5172
Name: count, dtype: int64

In [18]:
df_competencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5172 entries, 0 to 14998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       5172 non-null   object        
 1   id_cliente       5172 non-null   object        
 2   nombre           5172 non-null   object        
 3   apellido         5172 non-null   object        
 4   mail             5172 non-null   object        
 5   competencia      5172 non-null   bool          
 6   fecha_reserva    0 non-null      datetime64[ns]
 7   inicio_estancia  5149 non-null   datetime64[ns]
 8   final_estancia   5149 non-null   datetime64[ns]
 9   id_hotel         5172 non-null   int64         
 10  precio_noche     0 non-null      float64       
 11  nombre_hotel     5172 non-null   object        
 12  estrellas        0 non-null      Int64         
 13  ciudad           5172 non-null   object        
dtypes: Int64(1), bool(1), datetime64[ns](3), flo

In [19]:
##consigo los datos faltantes con el scrapeo. Detalle en el documento scrapeo.ipynb

# Análisis columna id_cliente

In [20]:
len(df.id_cliente.unique())

14847

In [21]:
len(df.mail.unique())

14905

In [22]:
df["nombre_apellido"] = df['nombre'].str.lower() +'.' +df['apellido'].str.lower()

In [23]:
len(df[df['competencia']== False]['nombre_apellido'].unique())

9785

In [24]:
#tengo menos id_clientes que combinaciones de nombres y apellidos. Parece que hay clientes con un mismo id y diferentes nombres. Intento localizarlos

df["id_nombre_apellido"] = df["id_cliente"]+"|"+df["nombre_apellido"]
df
df_id_nombre_apellido =df["id_nombre_apellido"].reset_index()
df_id_nombre_apellido

,index,id_nombre_apellido
0,0,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...
1,1,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....
2,2,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...
3,3,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet
4,4,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...
...,...,...
14995,14995,de729428-ec23-4665-a5c9-79e3ec282470|lorena.román
14996,14996,8469b819-d96a-4904-9cdd-710764f6301f|toño.narváez
14997,14997,e89751f4-3621-4636-81d3-90e2baa709cb|nadia.ver...
14998,14998,c4117358-fd89-44cd-b388-4ec6d3538c72|custodia....


In [25]:
df_id_nombre_apellido['id']=df['id_nombre_apellido'].str.split('|', expand=True)[0]
df_id_nombre_apellido['nombre_apellido']=df['id_nombre_apellido'].str.split('|', expand=True)[1]

In [26]:
df_contador_nombres_apellidos = df_id_nombre_apellido.nombre_apellido.value_counts().reset_index()
df_contador_nombres_apellidos

,nombre_apellido,count
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
14900,plácido.ocaña,1
14901,agapito.ortiz,1
14902,eligio.paredes,1
14903,roxana.teruel,1


In [27]:
df_contador_nombres_apellidos[df_contador_nombres_apellidos['count']>1]

,nombre_apellido,count
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
88,brígida.cózar,2
89,sofía.pellicer,2
90,florencia.olivé,2
91,olalla.lillo,2


In [28]:
lista_nombres_duplicados= df_contador_nombres_apellidos[df_contador_nombres_apellidos['count']>1].nombre_apellido.to_list()

In [29]:
df[df['nombre_apellido']=='victoriano.ródenas']

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido
4695,264e8d8a-cf96-40e6-b39d-88a857840ed9,883c41b0-0936-4bb0-b5aa-c4c85b52108a,Victoriano,Ródenas,victoriano.ródenas@example.com,False,2025-02-05,2025-03-01,2025-03-02,3,167.32,Hotel Monte Verde,2,Madrid,victoriano.ródenas,883c41b0-0936-4bb0-b5aa-c4c85b52108a|victorian...
6291,5f4d7519-6a4f-4a62-98ab-f164ccd8e1e1,6aadb18c-9dac-471a-bb1c-66c563ac4285,Victoriano,Ródenas,victoriano.ródenas@example.com,True,NaT,2025-03-01,2025-03-02,181,NaN,,<NA>,,victoriano.ródenas,6aadb18c-9dac-471a-bb1c-66c563ac4285|victorian...
12406,bcd8c15f-6190-497a-9031-873f9f34c950,c8b8e728-f11f-4001-ba70-f11ca5b20cce,Victoriano,Ródenas,victoriano.ródenas@example.com,False,2025-02-10,2025-03-01,2025-03-02,29,392.54,Hotel Monte Verde,5,Madrid,victoriano.ródenas,c8b8e728-f11f-4001-ba70-f11ca5b20cce|victorian...


In [30]:
dict_clientes_dup ={'nombre_completo':[],
                     'num_id_duplicados':[]}
for nombre in lista_nombres_duplicados:
    nombre_completo=nombre
    num_ids_duplicados = len(df[df.nombre_apellido == nombre].id_cliente.value_counts())
    dict_clientes_dup['nombre_completo'].append(nombre_completo)
    dict_clientes_dup['num_id_duplicados'].append(num_ids_duplicados)
dict_clientes_dup
df_clientes_dup=pd.DataFrame(dict_clientes_dup)
df_clientes_dup

,nombre_completo,num_id_duplicados
0,victoriano.ródenas,3
1,ariadna.miguel,3
2,manuelita.leiva,2
3,anselma.cano,2
4,sergio.carrión,2
...,...,...
88,brígida.cózar,2
89,sofía.pellicer,2
90,florencia.olivé,2
91,olalla.lillo,2


In [31]:
df_contador_ids = df_id_nombre_apellido.id.value_counts().reset_index()
df_contador_ids

,id,count
0,8bec1ff1-84d7-4cad-937f-dbae37691569,3
1,ba60cff3-280a-4948-b6da-5e76e57b9d79,3
2,f0c7fbf6-cea3-42fc-a94f-11c69dbf5713,3
3,3369b2b1-6b9a-4914-b7bb-655dd1d9c90f,2
4,d5897c77-6f76-4830-a64f-f6074f4833d1,2
...,...,...
14842,697fb6ac-a6bd-4297-929c-2e2a287078ff,1
14843,fd5ad4dc-9b25-4087-86d9-af3658d0333b,1
14844,f0bb12f5-4649-4dfe-9562-2edbc303338c,1
14845,90190fd5-4ecc-4b3e-b56c-47f1daa81463,1


In [32]:
df[df['id_cliente']=='8bec1ff1-84d7-4cad-937f-dbae37691569']

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido
1482,83a9686e-1e15-4cf1-9092-7c31e39db089,8bec1ff1-84d7-4cad-937f-dbae37691569,Raquel,Jara,raquel.jara@example.com,False,2025-02-01,2025-03-01,2025-03-02,36,399.29,Hotel Luz de Madrid,3,Madrid,raquel.jara,8bec1ff1-84d7-4cad-937f-dbae37691569|raquel.jara
2287,cede56a0-4525-48d4-966a-9bf94a10b4f0,8bec1ff1-84d7-4cad-937f-dbae37691569,Manolo,Crespo,manolo.crespo@example.com,False,2025-02-10,2025-03-01,2025-03-02,48,299.60,Hotel Vista Alegre,3,Madrid,manolo.crespo,8bec1ff1-84d7-4cad-937f-dbae37691569|manolo.cr...
6764,eae82e30-181e-4931-883e-9bcb55ce2125,8bec1ff1-84d7-4cad-937f-dbae37691569,Tito,Mir,tito.mir@example.com,True,NaT,2025-03-01,2025-03-02,135,NaN,,<NA>,,tito.mir,8bec1ff1-84d7-4cad-937f-dbae37691569|tito.mir


In [33]:
lista_ids_duplicados = df_contador_ids[df_contador_ids['count']>1].id.to_list()

In [34]:
dict_ids_dup ={'id':[],
                     'num_clientes_duplicados':[]}
for id in lista_ids_duplicados:
    ids=id
    num_clientes_duplicados = len(df[df.id_cliente == id].nombre_apellido.value_counts())
    dict_ids_dup['id'].append(ids)
    dict_ids_dup['num_clientes_duplicados'].append(num_clientes_duplicados)
dict_ids_dup
df_ids_dup=pd.DataFrame(dict_ids_dup)
df_ids_dup

,id,num_clientes_duplicados
0,8bec1ff1-84d7-4cad-937f-dbae37691569,3
1,ba60cff3-280a-4948-b6da-5e76e57b9d79,3
2,f0c7fbf6-cea3-42fc-a94f-11c69dbf5713,3
3,3369b2b1-6b9a-4914-b7bb-655dd1d9c90f,2
4,d5897c77-6f76-4830-a64f-f6074f4833d1,2
...,...,...
145,7484177b-51d6-44b0-9ffb-d052fa2eb95b,2
146,70e63c2b-1eda-4df2-8bea-66872b0a1807,2
147,8e66b4d1-95b8-409f-b806-8d1ac0fabcd4,2
148,99e2b670-a1cc-470c-a41f-10464e894e33,2


In [35]:
len(df.nombre_apellido.unique()) == len(df.mail.unique()) 

True

In [36]:
#Hay tanto clientes con ids duplicados como ids asignados a diferentes clientes. Por tanto, creo un id para cada cliente basandome la unión del nombre y apellido que es lo mismo que el correo electronico
df.nombre_apellido.unique()
df_clientes_unicos = pd.DataFrame(df.nombre_apellido.unique())
df_clientes_unicos['id_cliente_unico'] = None
for indice in df_clientes_unicos.index:
    df_clientes_unicos.iloc[indice,1] = 'C'+str(indice)+df_clientes_unicos.iloc[indice,0].split('.')[0][0]+df_clientes_unicos.iloc[indice,0].split('.')[1][0]
df_clientes_unicos=df_clientes_unicos.set_axis(['nombre_apellido', 'id_cliente_unico'], axis=1)
df_clientes_unicos


,nombre_apellido,id_cliente_unico
0,feliciana.cantón,C0fc
1,leonardo.hierro,C1lh
2,maite.calatayud,C2mc
3,tecla.bonet,C3tb
4,amílcar.andrés,C4aa
...,...,...
14900,lorena.román,C14900lr
14901,toño.narváez,C14901tn
14902,nadia.verdugo,C14902nv
14903,custodia.marco,C14903cm


In [37]:
df_filtrado = df_clientes_unicos[df_clientes_unicos['nombre_apellido']=='leonardo.hierro']
df_filtrado.loc[:,'id_cliente_unico'].values[0]

'C1lh'

In [38]:
df['id_cliente_unico'] = None
for i in df.index:
    nombre_completo = df['nombre_apellido'][i]
    id_cliente_unico_1 = df_clientes_unicos[df_clientes_unicos['nombre_apellido']==nombre_completo].id_cliente_unico.values[0]
    df.loc[i,'id_cliente_unico']=id_cliente_unico_1
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido,id_cliente_unico
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,<NA>,,feliciana.cantón,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...,C0fc
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,<NA>,,leonardo.hierro,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....,C1lh
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1,Madrid,maite.calatayud,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...,C2mc
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5,Madrid,tecla.bonet,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet,C3tb
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1,Madrid,amílcar.andrés,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...,C4aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,58acd6f3-7160-4973-b951-b14188937444,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com,False,2025-02-06,2025-03-01,2025-03-02,35,111.78,Hotel Jardines del Rey,1,Madrid,lorena.román,de729428-ec23-4665-a5c9-79e3ec282470|lorena.román,C14900lr
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,<NA>,,toño.narváez,8469b819-d96a-4904-9cdd-710764f6301f|toño.narváez,C14901tn
14997,af0f6afa-e7d2-410a-8cdd-5eb62b1e4788,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com,False,2025-02-11,2025-03-01,2025-03-02,47,442.96,Hotel Brisas del Mar,5,Madrid,nadia.verdugo,e89751f4-3621-4636-81d3-90e2baa709cb|nadia.ver...,C14902nv
14998,6d1f590c-ae87-4582-bc99-9b8cbf1d8e28,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,<NA>,,custodia.marco,c4117358-fd89-44cd-b388-4ec6d3538c72|custodia....,C14903cm


# Columnas inicio_estancia y final_estancia

In [39]:
#las 2 columnas tienen valores nulos. Los relleno con el único valor existente:
df.inicio_estancia.value_counts()
df["inicio_estancia"] = df["inicio_estancia"][0]
df["final_estancia"] =  df["final_estancia"][0]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_reserva          15000 non-null  object        
 1   id_cliente          15000 non-null  object        
 2   nombre              15000 non-null  object        
 3   apellido            15000 non-null  object        
 4   mail                15000 non-null  object        
 5   competencia         15000 non-null  bool          
 6   fecha_reserva       9828 non-null   datetime64[ns]
 7   inicio_estancia     15000 non-null  datetime64[ns]
 8   final_estancia      15000 non-null  datetime64[ns]
 9   id_hotel            15000 non-null  int64         
 10  precio_noche        9776 non-null   float64       
 11  nombre_hotel        15000 non-null  object        
 12  estrellas           9828 non-null   Int64         
 13  ciudad              15000 non-null  object        


# Columna fecha_reserva

In [40]:
df.fecha_reserva.value_counts()
#Totas las reservas se han realizado en febrero de 2025

fecha_reserva
2025-02-06    872
2025-02-04    871
2025-02-02    853
2025-02-03    833
2025-02-05    825
2025-02-09    820
2025-02-01    814
2025-02-10    812
2025-02-12    811
2025-02-11    775
2025-02-07    775
2025-02-08    767
Name: count, dtype: int64

In [41]:
df_fecha_reserva_nulos = df[df['fecha_reserva'].isnull()]
df_fecha_reserva_nulos[df_fecha_reserva_nulos.competencia==False]
#Todos los nulos que tiene son de reservas de la competencia. ¿Qué valor le doy? ¿Me lo invento? Hay que darle la fecha en la que se realiza el escrapeo

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido,id_cliente_unico


# Columna id_hotel y nombre_hotel 

In [42]:
len(df.id_hotel.value_counts())

29

In [43]:
df_no_competencia = df[df.competencia == False]

In [44]:
len(df_no_competencia.id_hotel.value_counts())

19

In [45]:
df.id_hotel.value_counts()

id_hotel
28     559
6      543
36     542
13     541
114    537
27     536
38     534
103    533
2      531
128    530
14     528
131    524
186    522
48     519
135    518
40     516
113    515
15     514
194    511
19     511
33     508
49     506
35     506
3      501
47     498
181    493
117    489
43     485
29     450
Name: count, dtype: int64

In [46]:
len(df.nombre_hotel .value_counts())

20

In [47]:
df.nombre_hotel.value_counts()

nombre_hotel
                          5172
Hotel Sol y Luna           557
Hotel Monte Verde          550
Hotel Luz de Madrid        536
Hotel Brisas del Mar       534
Hotel Mirador Real         533
Hotel Los Almendros        533
Hotel Puerta del Cielo     526
Hotel Camino del Sol       518
Gran Hotel Madrid          518
Hotel Costa Azul           515
Hotel Las Estrellas        514
Hotel Torre Dorada         512
Hotel Rincón Sereno        512
Hotel Vista Alegre         511
Hotel Encanto Real         511
Palacio del Sol            510
Hotel Jardines del Rey     502
Hotel Maravilla Real       473
Hotel Palacio Imperial     463
Name: count, dtype: int64

In [48]:
#Creo una columna para combinar el id y el nombre para ver cuantos elementos únicos me aparecen ya que no coincide la cantidad existente en los ids con la cantidad existente en los nombre
df['id_nombre_hotel']= df['id_hotel'].astype('str') + '|' + df['nombre_hotel']

In [49]:
len(df['id_nombre_hotel'].value_counts())
print(f'Existen {len(df['id_nombre_hotel'].value_counts())} combinaciones diferentes')

Existen 371 combinaciones diferentes


In [50]:
lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
lista_nombre_hoteles[1:]

['Hotel Monte Verde',
 'Hotel Brisas del Mar',
 'Hotel Camino del Sol',
 'Hotel Puerta del Cielo',
 'Hotel Encanto Real',
 'Palacio del Sol',
 'Hotel Jardines del Rey',
 'Hotel Las Estrellas',
 'Gran Hotel Madrid',
 'Hotel Torre Dorada',
 'Hotel Palacio Imperial',
 'Hotel Luz de Madrid',
 'Hotel Los Almendros',
 'Hotel Sol y Luna',
 'Hotel Mirador Real',
 'Hotel Rincón Sereno',
 'Hotel Vista Alegre',
 'Hotel Costa Azul',
 'Hotel Maravilla Real']

In [51]:
lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
lista_ids_hoteles

[14, 38, 35, 36, 43, 6, 3, 29, 47, 27, 13, 28, 40, 2, 48, 15, 33, 49, 19]

In [52]:
#cada nombre de hotel tiene todos los ids posibles. Hay que reestructurar los ids
for hotel in lista_nombre_hoteles[1:]:
    df[df.nombre_hotel==hotel].id_hotel.value_counts().reset_index().iloc[0]
    print(hotel)
    print(df[df.nombre_hotel==hotel].id_hotel.value_counts().reset_index().iloc[0])

Hotel Monte Verde
id_hotel     6
count       41
Name: 0, dtype: int64
Hotel Brisas del Mar
id_hotel    27
count       38
Name: 0, dtype: int64
Hotel Camino del Sol
id_hotel    15
count       36
Name: 0, dtype: int64
Hotel Puerta del Cielo
id_hotel    28
count       37
Name: 0, dtype: int64
Hotel Encanto Real
id_hotel     2
count       35
Name: 0, dtype: int64
Palacio del Sol
id_hotel     6
count       42
Name: 0, dtype: int64
Hotel Jardines del Rey
id_hotel     6
count       33
Name: 0, dtype: int64
Hotel Las Estrellas
id_hotel     2
count       38
Name: 0, dtype: int64
Gran Hotel Madrid
id_hotel    38
count       43
Name: 0, dtype: int64
Hotel Torre Dorada
id_hotel    33
count       37
Name: 0, dtype: int64
Hotel Palacio Imperial
id_hotel    38
count       32
Name: 0, dtype: int64
Hotel Luz de Madrid
id_hotel    15
count       37
Name: 0, dtype: int64
Hotel Los Almendros
id_hotel    15
count       38
Name: 0, dtype: int64
Hotel Sol y Luna
id_hotel    36
count       40
Name: 0, dtype: 

In [53]:
#No existe un patrón en el código anterior, hay ids que aparecen varias veces como los más repetidos. Por tanto, creo un diccionario nuevo para darle un id a cada hotel según el nombre:
lista_nombre_hoteles = df.nombre_hotel.unique().tolist()
lista_ids_hoteles = df_no_competencia.id_hotel.unique().tolist()
dict_id_nombre_hotel = {'id_hotel': [], 
                        'nombre_hotel': []}
for hotel in lista_nombre_hoteles[1:]:
    dict_id_nombre_hotel["nombre_hotel"].append(hotel)
for id in lista_ids_hoteles:
    dict_id_nombre_hotel["id_hotel"].append(id)
dict_id_nombre_hotel

df_id_hoteles_nombre_hoteles = pd.DataFrame(dict_id_nombre_hotel)
df_id_hoteles_nombre_hoteles

##genero esta tabla para los ids de hoteles, pero yo reiniciaria los ids al llevarmelos a la BBDD 

,id_hotel,nombre_hotel
0,14,Hotel Monte Verde
1,38,Hotel Brisas del Mar
2,35,Hotel Camino del Sol
3,36,Hotel Puerta del Cielo
4,43,Hotel Encanto Real
5,6,Palacio del Sol
6,3,Hotel Jardines del Rey
7,29,Hotel Las Estrellas
8,47,Gran Hotel Madrid
9,27,Hotel Torre Dorada


In [54]:
#sustituyo los ids correctos en el df original
keys= list(df_id_hoteles_nombre_hoteles.nombre_hotel)
values = df_id_hoteles_nombre_hoteles.id_hotel.to_list()
diccionario_nombre_hoteles_id_hoteles =dict(zip(keys,values))
df.loc[df['competencia']==False, 'id_hotel'] = df['nombre_hotel'].map(diccionario_nombre_hoteles_id_hoteles)
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido,id_cliente_unico,id_nombre_hotel
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,<NA>,,feliciana.cantón,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...,C0fc,113|
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,<NA>,,leonardo.hierro,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....,C1lh,194|
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1,Madrid,maite.calatayud,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...,C2mc,14|Hotel Monte Verde
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5,Madrid,tecla.bonet,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet,C3tb,38|Hotel Brisas del Mar
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1,Madrid,amílcar.andrés,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...,C4aa,35|Hotel Camino del Sol


# Columna estrellas

In [55]:
df.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad',
       'nombre_apellido', 'id_nombre_apellido', 'id_cliente_unico',
       'id_nombre_hotel'],
      dtype='object')

In [56]:
#Cada hotel tiene diferentes estrellas. Por ejemplo: 
df[df.nombre_hotel == 'Palacio del Sol'].estrellas.value_counts()
#Calculo las estrellas para cada hotel
df_estrellas = df_no_competencia.groupby('nombre_hotel')['estrellas'].mean().round(1)
df_estrellas= df_estrellas.reset_index()
df_estrellas

,nombre_hotel,estrellas
0,Gran Hotel Madrid,3.1
1,Hotel Brisas del Mar,3.1
2,Hotel Camino del Sol,3.1
3,Hotel Costa Azul,3.1
4,Hotel Encanto Real,3.0
5,Hotel Jardines del Rey,2.9
6,Hotel Las Estrellas,2.9
7,Hotel Los Almendros,3.0
8,Hotel Luz de Madrid,3.1
9,Hotel Maravilla Real,3.0


In [57]:
# Relleno la información de la valoración de los hoteles propios con la media de las estrellas calculadas:
keys= list(df_estrellas.nombre_hotel)
values = df_estrellas.estrellas.to_list()
diccionario_valoración_hoteles_propios =dict(zip(keys,values))
df.loc[df['competencia']==False, 'valoracion'] = df['nombre_hotel'].map(diccionario_valoración_hoteles_propios)
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido,id_cliente_unico,id_nombre_hotel,valoracion
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,<NA>,,feliciana.cantón,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...,C0fc,113|,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,<NA>,,leonardo.hierro,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....,C1lh,194|,NaN
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1,Madrid,maite.calatayud,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...,C2mc,14|Hotel Monte Verde,3.1
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5,Madrid,tecla.bonet,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet,C3tb,38|Hotel Brisas del Mar,3.1
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1,Madrid,amílcar.andrés,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...,C4aa,35|Hotel Camino del Sol,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,58acd6f3-7160-4973-b951-b14188937444,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com,False,2025-02-06,2025-03-01,2025-03-02,3,111.78,Hotel Jardines del Rey,1,Madrid,lorena.román,de729428-ec23-4665-a5c9-79e3ec282470|lorena.román,C14900lr,35|Hotel Jardines del Rey,2.9
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,<NA>,,toño.narváez,8469b819-d96a-4904-9cdd-710764f6301f|toño.narváez,C14901tn,114|,NaN
14997,af0f6afa-e7d2-410a-8cdd-5eb62b1e4788,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com,False,2025-02-11,2025-03-01,2025-03-02,38,442.96,Hotel Brisas del Mar,5,Madrid,nadia.verdugo,e89751f4-3621-4636-81d3-90e2baa709cb|nadia.ver...,C14902nv,47|Hotel Brisas del Mar,3.1
14998,6d1f590c-ae87-4582-bc99-9b8cbf1d8e28,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,<NA>,,custodia.marco,c4117358-fd89-44cd-b388-4ec6d3538c72|custodia....,C14903cm,194|,NaN


# Columna ciudad

In [58]:
df.ciudad.value_counts()


ciudad
Madrid    9828
          5172
Name: count, dtype: int64

In [59]:
#Todos los que no tienen valores son los hoteles de la competencia. Igualmente la ciudad será Madrid, pero lo rellenaré cuando haga el escrapeo.
df_competencia.ciudad.value_counts()

ciudad
    5172
Name: count, dtype: int64

# Columna precios

In [60]:
df_no_competencia.precio_noche.isnull().sum()
print(f'Tengo {df_no_competencia.precio_noche.isnull().sum()} nulos en la columna precio_noche')

Tengo 52 nulos en la columna precio_noche


In [61]:
df_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].groupby(['nombre_hotel', 'fecha_reserva'])['id_reserva'].count().reset_index()
df_precios_nulos_hoteles_propios.iloc[3,:]

nombre_hotel     Hotel Brisas del Mar
fecha_reserva     2025-02-11 00:00:00
id_reserva                          1
Name: 3, dtype: object

In [62]:
len(df_precios_nulos_hoteles_propios)

48

In [63]:
df_precios_nulos_hoteles_propios.head(1)

,nombre_hotel,fecha_reserva,id_reserva
0,Gran Hotel Madrid,2025-02-02,1


In [64]:
#Calculo los outliers de cada df filtrado por el nombre del hotel:
lista_hoteles_propios = df_no_competencia.nombre_hotel.values
def detectar_oulieres_iqr(dataframe):
    resultados = {}
    for nombrehotel in lista_hoteles_propios:
        df_filtrado = dataframe[dataframe['nombre_hotel']== nombrehotel]
        Q1=df_filtrado['precio_noche'].quantile(0.25) #Primer cuartil
        Q3 = df_filtrado['precio_noche'].quantile(0.75) #Tercer cuartil
        IQR = Q3-Q1 #rango intercuartil

        #Defino los limites de los outliers
        lower_bound = Q1-1.5*IQR
        upper_bound = Q3+1.5*IQR

        #Filtrar los valores outliers
        outliers = df_filtrado[(df_filtrado['precio_noche'] < lower_bound) | (df_filtrado['precio_noche'] > upper_bound)]
        num_outliers = len(outliers)
        total_values = len(df_filtrado)
        percentage_outliers = (num_outliers/total_values)*100

        #Guardo los resultados
        resultados[nombrehotel] = {'Cantidad de Outliers': num_outliers,
                                                            'Porcentaje de Outliers': round(percentage_outliers,2)
                                                            }
    return resultados


In [65]:
resultados = detectar_oulieres_iqr(df_no_competencia)
pd.DataFrame(resultados).T

,Cantidad de Outliers,Porcentaje de Outliers
Hotel Monte Verde,0.0,0.0
Hotel Brisas del Mar,0.0,0.0
Hotel Camino del Sol,0.0,0.0
Hotel Puerta del Cielo,0.0,0.0
Hotel Encanto Real,0.0,0.0
Palacio del Sol,0.0,0.0
Hotel Jardines del Rey,0.0,0.0
Hotel Las Estrellas,0.0,0.0
Gran Hotel Madrid,0.0,0.0
Hotel Torre Dorada,0.0,0.0


In [66]:
#Como no hay outliers calculo el precio medio de la noche teniendo en cuenta tanto el nombre como la fecha de la reserva:
df_precios_nulos_hoteles_propios["precio_noche_medio"] = None
for indice in range(0,len(df_precios_nulos_hoteles_propios)):
    nombre_hotel= df_precios_nulos_hoteles_propios.iloc[indice,:].nombre_hotel
    fecha_reserva=df_precios_nulos_hoteles_propios.iloc[indice,:].fecha_reserva
    df_precios_nulos_hoteles_propios.iloc[indice,3] = df[df.nombre_hotel == nombre_hotel][df[df.nombre_hotel == nombre_hotel].fecha_reserva == fecha_reserva].precio_noche.mean().round(2)
df_precios_nulos_hoteles_propios

,nombre_hotel,fecha_reserva,id_reserva,precio_noche_medio
0,Gran Hotel Madrid,2025-02-02,1,273.4
1,Gran Hotel Madrid,2025-02-06,1,297.75
2,Gran Hotel Madrid,2025-02-09,1,257.58
3,Hotel Brisas del Mar,2025-02-11,1,275.04
4,Hotel Brisas del Mar,2025-02-12,1,274.04
5,Hotel Camino del Sol,2025-02-06,2,236.67
6,Hotel Camino del Sol,2025-02-08,1,262.23
7,Hotel Camino del Sol,2025-02-11,1,221.31
8,Hotel Costa Azul,2025-02-01,1,267.59
9,Hotel Costa Azul,2025-02-08,1,304.86


In [67]:
#Indice de los que tienen precios nulos y son hoteles propios
lista_indices_precios_nulos_hoteles_propios = df_no_competencia[df_no_competencia.precio_noche.isnull()].index
lista_indices_precios_nulos_hoteles_propios

Index([  351,   411,   487,   884,  1623,  2020,  2459,  2543,  3035,  3910,
        4123,  4604,  4950,  5030,  5584,  5697,  5970,  6179,  6315,  6836,
        7103,  7698,  8290,  8735,  8788,  9022,  9132,  9205,  9263,  9365,
        9367,  9420,  9548,  9672,  9753,  9943,  9972, 10057, 10144, 10385,
       10691, 11087, 11366, 11848, 12159, 12211, 12540, 12837, 13778, 14293,
       14361, 14448],
      dtype='int64')

In [68]:
df.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad',
       'nombre_apellido', 'id_nombre_apellido', 'id_cliente_unico',
       'id_nombre_hotel', 'valoracion'],
      dtype='object')

In [69]:
#Sustituyo los precios que falta en el DF original teniendo en cuenta los precios medios calculados:
for indice in lista_indices_precios_nulos_hoteles_propios:
    fila = df.iloc[indice,:]
    nombre_hotel = fila['nombre_hotel']
    fecha_reserva = fila['fecha_reserva']
    precio_medio = df_precios_nulos_hoteles_propios[(df_precios_nulos_hoteles_propios['nombre_hotel'] == nombre_hotel) & (df_precios_nulos_hoteles_propios['fecha_reserva'] == fecha_reserva)].precio_noche_medio
    df.iloc[indice,10]=precio_medio


## Relleno los datos faltantes en el DF de la competencia conseguidas con el escrapeo

In [70]:
FICHERO_ESCRAPEADO= os.getenv("ARCHIVO_GUARDAR_ESCRAPEO")
df_escrapeo = pd.read_pickle(FICHERO_ESCRAPEADO)
df_escrapeo.columns

Index(['nombre_hotel', 'rating', 'precio_noche', 'fecha_escrapeo', 'ciudad'], dtype='object')

In [71]:
values = list(df_competencia.id_hotel.drop_duplicates())
keys = list(df_escrapeo.nombre_hotel.unique())

#Creo un diccionario:
diccionario_idhotel_nombrehotel = dict(zip(keys,values))
diccionario_idhotel_nombrehotel

{'Novotel Madrid Center': 113,
 'ibis budget Madrid Calle 30': 194,
 'ibis Madrid Centro las Ventas': 131,
 'ibis budget Madrid Centro las Ventas': 114,
 'Novotel Madrid City Las Ventas': 103,
 'ibis Madrid Calle Alcalá': 181,
 'ibis budget Madrid Vallecas': 128,
 'Pullman Madrid Airport & Feria': 186,
 'Novotel Madrid Campo de las Naciones': 135,
 'ibis Madrid Aeropuerto Barajas': 117}

In [72]:
df_escrapeo_completo = []
for index,row in df_escrapeo.iterrows():
    nombre_hotel =row["nombre_hotel"]
    id_hotel = diccionario_idhotel_nombrehotel.get(nombre_hotel)
    valoracion = row["rating"]
    precio_noche = row["precio_noche"]
    fecha_escrapeo= row["fecha_escrapeo"]
    ciudad = row["ciudad"]
    df_escrapeo_completo.append([id_hotel,nombre_hotel,valoracion,precio_noche,fecha_escrapeo,ciudad])
df_escrapeo_completo= pd.DataFrame(df_escrapeo_completo, columns=['id_hotel','nombre_hotel', 'valoracion', 'precio_noche', 'fecha_reserva', 'ciudad'])
df_escrapeo_completo

,id_hotel,nombre_hotel,valoracion,precio_noche,fecha_reserva,ciudad
0,113,Novotel Madrid Center,4.6,268,2025-02-25,Madrid
1,194,ibis budget Madrid Calle 30,4.4,110,2025-02-25,Madrid
2,131,ibis Madrid Centro las Ventas,4.5,181,2025-02-25,Madrid
3,114,ibis budget Madrid Centro las Ventas,4.3,119,2025-02-25,Madrid
4,103,Novotel Madrid City Las Ventas,4.6,182,2025-02-25,Madrid
5,181,ibis Madrid Calle Alcalá,4.5,133,2025-02-25,Madrid
6,128,ibis budget Madrid Vallecas,4.3,107,2025-02-25,Madrid
7,186,Pullman Madrid Airport & Feria,4.2,143,2025-02-25,Madrid
8,135,Novotel Madrid Campo de las Naciones,4.5,162,2025-02-25,Madrid
9,117,ibis Madrid Aeropuerto Barajas,4.4,129,2025-02-25,Madrid


In [73]:
for columna in df_escrapeo_completo.columns[1:]:
    keys = list(df_escrapeo_completo.id_hotel.drop_duplicates())
    values = list(df_escrapeo_completo[columna].values)
    diccionario_idhotel_columna = dict(zip(keys,values))
    df.loc[df['competencia'], columna] = df['id_hotel'].map(diccionario_idhotel_columna)
df.head()


C:\Users\van-2\AppData\Local\Temp\ipykernel_1284\750668825.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['268' '110' '181' ... '143' '119' '110']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['competencia'], columna] = df['id_hotel'].map(diccionario_idhotel_columna)


,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,nombre_apellido,id_nombre_apellido,id_cliente_unico,id_nombre_hotel,valoracion
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-25,2025-03-01,2025-03-02,113,268,Novotel Madrid Center,<NA>,Madrid,feliciana.cantón,346f2a77-33f4-4b2d-82f3-e8501e21b28d|feliciana...,C0fc,113|,4.6
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-25,2025-03-01,2025-03-02,194,110,ibis budget Madrid Calle 30,<NA>,Madrid,leonardo.hierro,346f2a77-33f4-4b2d-82f3-e8501e21b28d|leonardo....,C1lh,194|,4.4
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1,Madrid,maite.calatayud,6ec4d7d2-e382-47cc-94f6-c647577d4035|maite.cal...,C2mc,14|Hotel Monte Verde,3.1
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5,Madrid,tecla.bonet,aed17fe8-eb18-4f69-bf08-df1afdc339c3|tecla.bonet,C3tb,38|Hotel Brisas del Mar,3.1
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1,Madrid,amílcar.andrés,710a2fbb-264f-426a-aa66-f8fffd146a2d|amílcar.a...,C4aa,35|Hotel Camino del Sol,3.1


## Elimino las columnas que no necesito:

In [74]:
df.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad',
       'nombre_apellido', 'id_nombre_apellido', 'id_cliente_unico',
       'id_nombre_hotel', 'valoracion'],
      dtype='object')

In [75]:
df.drop(columns=['id_cliente', 'estrellas','id_nombre_apellido','nombre_apellido', 'id_nombre_hotel'],inplace=True)


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_reserva        15000 non-null  object        
 1   nombre            15000 non-null  object        
 2   apellido          15000 non-null  object        
 3   mail              15000 non-null  object        
 4   competencia       15000 non-null  bool          
 5   fecha_reserva     15000 non-null  datetime64[ns]
 6   inicio_estancia   15000 non-null  datetime64[ns]
 7   final_estancia    15000 non-null  datetime64[ns]
 8   id_hotel          15000 non-null  int64         
 9   precio_noche      15000 non-null  object        
 10  nombre_hotel      15000 non-null  object        
 11  ciudad            15000 non-null  object        
 12  id_cliente_unico  15000 non-null  object        
 13  valoracion        15000 non-null  float64       
dtypes: bool(1), datetime64[ns](

In [77]:
df_limpio = df.to_pickle(ARCHIVO_SALIDA)